In [1]:
#https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews/data
#https://www.kaggle.com/code/stpeteishii/book-review-text-to-summary-bart-fit-predict

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
import nltk
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())
import importlib
import re
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.tensorboard import SummaryWriter
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report

/Lab1


2025-04-14 15:25:21.799313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744637121.833296   56008 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744637121.844370   56008 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744637121.872261   56008 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744637121.872291   56008 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744637121.872295   56008 computation_placer.cc:177] computation placer alr

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the Feedforward Neural Network
class SentimentClassifier(nn.Module):
    def __init__(self, input_size, hidden_size = 128, output_size = 6):
        super(SentimentClassifier, self).__init__()
        self.inlayer = nn.Linear(input_size, 2*hidden_size)   # First hidden layer
        self.hidden1 = nn.Linear(2*hidden_size, hidden_size)          # Output layer (0 to 5 stars)
        self.hidden2 = nn.Linear(hidden_size, hidden_size // 2)
        self.hidden3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.outlayer = nn.Linear(hidden_size // 4, output_size)          # Second hidden layer 
        self.droput5 = nn.Dropout()                                # Dropout layer (50%)
        self.droput3 = nn.Dropout(.3)
    def forward(self, x):
        x = F.relu(self.inlayer(x))
        x = self.droput5(x)
        x = F.relu(self.hidden1(x))
        x = self.droput3(x)
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.outlayer(x)

        return x


In [4]:
data=pd.read_csv('Books_rating.csv')
print(len(data))
display(data[0:1].T)
data=data[0:100000]

3000000


,0
Id,1882931173
Title,Its Only Art If Its Well Hung!
Price,NaN
User_id,AVCGYZL8FQQTD
profileName,"Jim of Oz ""jim-of-oz"""
review/helpfulness,7/7
review/score,4.0
review/time,940636800
review/summary,Nice collection of Julie Strain images
review/text,This is only for Julie Strain fans. It's a col...


In [5]:
inputt=data['review/text']
labelt=data['review/score']

print(inputt[0])

This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paintings by Olivia.If you're looking for heavy literary content, this isn't the place to find it -- there's only about 2 pages with text and everything else is photos.Bottom line: if you only want one book, the Six Foot One ... is probably a better choice, however, if you like Julie like I like Julie, you won't go wrong on this one either.


In [6]:


training_data, temp_data, training_labels, temp_labels = train_test_split(
    inputt, labelt, test_size=0.2, random_state=42
)
validation_data, test_data, validation_labels, test_labels = train_test_split(
    temp_data, temp_labels, test_size=0.5, random_state=42
)

In [ ]:
for i, text in enumerate(inputt):
    if not isinstance(text, str):  # Convert non-strings to empty string
        text = str(text) if text is not None else ''
    
    text = text.lower()
    text = re.sub(r'[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', text)  # Remove emails
    text = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', text)  # Remove IPs
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\d+(\.\d+)?', '', text) #Remove numbers
    inputt[i] = text  # Update the list with cleaned text

labelt = [round(x) for x in labelt]


print(labelt[0])
print(inputt[0])

/tmp/ipykernel_56008/1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
/tmp/ipykernel_56008/1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
/tmp/ipykernel_56008/1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
/tmp/ipykernel_56008/1

In [ ]:
# vectorize data using TFIDF and transform for PyTorch for scalability
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
training_data = training_data.todense()                             # convert to dense matrix for Pytorch
vocab_size = len(word_vectorizer.vocabulary_)
validation_data = word_vectorizer.transform(validation_data)
validation_data = validation_data.todense()
train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()
test_data = word_vectorizer.transform(test_data)
test_data = test_data.todense()
test_x_tensor = torch.from_numpy(np.array(test_data)).type(torch.FloatTensor)
test_y_tensor = torch.from_numpy(np.array(test_labels)).long()

In [ ]:
# Create TensorDataset and DataLoader
batch_size = 32

train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(validation_x_tensor, validation_y_tensor)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(test_x_tensor, test_y_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define model, loss function, and optimizer
input_size = train_x_tensor.shape[1]  # TF-IDF vector size
print(input_size)
lr = 0.0001
model = SentimentClassifier(input_size)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)  # Optimizer

In [ ]:
num_epochs = 100  # Train for x amount of epochs

wandb.login()

wandb.init(project='sentiment-classification', config={
    'epochs': num_epochs,
    'batch_size': batch_size,
    'learning_rate': lr,
})
config = wandb.config


best_val_loss = float('inf')
best_model_state = None
patience = 10
epochs_no_improvement = 0
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_y = batch_y.to(device)
        optimizer.zero_grad()  # Clear gradients
        batch_x = batch_x.to(device)
        outputs = model(batch_x).to(device)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()
    

    model.eval()  # Set to evaluation mode
    total_val_loss = 0 
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient tracking for efficiency
        for batch_x, batch_y in validation_loader:
            batch_y = batch_y.to(device)
            batch_x = batch_x.to(device)
            outputs = model(batch_x).to(device)
            loss = criterion(outputs, batch_y)
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == batch_y).sum().item()
            total_val_loss += loss.item()
            total += batch_y.size(0)
            

    accuracy = correct / total
    wandb.log({
    'epoch': epoch,
    'train_loss': total_loss,
    'val_loss': total_val_loss,
    'val_accuracy': accuracy
    })
    
        
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Traning Loss: {total_loss/len(train_loader):.4f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {total_val_loss/len(validation_loader):.4f}")

    if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            best_model_state = model.state_dict()
            epochs_no_improvement = 0  # reset counter if there was an improvement
    else:
        epochs_no_improvement += 1  # increment counter if there was no improvement

    if epochs_no_improvement >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

In [ ]:
# Final evaluation on test set
model.load_state_dict(best_model_state)  # Load the best model
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Generate classification report
final_report = classification_report(all_labels, all_preds, output_dict=True)
wandb.log({"final_classification_report": final_report})
